# USAXS PLC Suspender simulation

Per https://github.com/APS-USAXS/ipython-usaxs/issues/82, simulate
the `plc_protect.stop_if_tripped` function as a bluesky suspender.

In [1]:
import epics
from ophyd import Device, Component, EpicsSignal
from bluesky import RunEngine
import bluesky.plans as bp
from bluesky.callbacks import LiveTable

In [2]:
class PlcProtectionDevice(Device):
    """
    Detector Protection PLC interface
    
    motion limit switches: 
    * SAXS_Y, WAXS_X, AX
    * zero when OFF
    * two limits must be ON to allow safe move of the third
    """
    SAXS_Y = Component(EpicsSignal, 'X11')
    WAXS_X = Component(EpicsSignal, 'X12')
    AX = Component(EpicsSignal, 'X13')
    
    operations_status = Component(EpicsSignal, 'Y0')     # 0=not good, 1=good
    
    SLEEP_POLL_s = 0.1
    _tripped_message = None

    tripped_text = """
    Equipment protection is engaged, no power on motors.
    Fix PLC protection before any move. Stopping now.
    Call beamline scientists if you do not understand.

    !!!!!!  DO NOT TRY TO FIX THIS YOURSELF  !!!!!!

    """
    suspender = None

    @property
    def interlocked(self):
        return not 0 in (self.SAXS_Y.value, self.WAXS_X.value, self.AX.value)
    
    def wait_for_interlock(self, verbose=True):
        t0 = time.time()
        msg = "Waiting %g for PLC interlock, check limit switches"
        while not self.interlocked:
            yield from bps.sleep(self.SLEEP_POLL_s)
            if verbose:
                print(msg % time.time()-t0)
    
    def stop_if_tripped(self, verbose=True):
        if self.operations_status.value == 1:
            self._tripped_message = None
        else:
            msg = self.tripped_text
            if verbose:
                print(msg)
            yield from bps.mv(
                ti_filter_shutter, "close",
                user_data.collection_in_progress, 0,     # notify the GUI and others
            )
            if self.suspender is not None:
                msg +="\n P.S. Can resume Bluesky scan: {}\n".format(
                    suspender.allow_resume)
            self._tripped_message = msg
    
    def stop_in_suspender(self):
        if self.operations_status.value == 1:
            msg = None
        else:
            msg = self.tripped_text
            if self.suspender is not None:
                msg +="\n P.S. Can resume Bluesky scan: {}\n".format(
                    self.suspender.allow_resume)
            ti_filter_shutter.close()
            user_data.collection_in_progress.put(0)     # notify the GUI and others
        return msg


In [3]:
class SimPlcProtectionDevice(PlcProtectionDevice):
    # for my simulator
    SAXS_Y = Component(EpicsSignal, 'otz:vars0:bit1')
    WAXS_X = Component(EpicsSignal, 'otz:vars0:bit2')
    AX = Component(EpicsSignal, 'otz:vars0:bit3')
    
    operations_status = Component(EpicsSignal, 'otz:vars0:bit0')     # 0=not good, 1=good
    
    def set_epics_names(self, signal, desc, znam, onam):
        pvname = signal.pvname
        epics.caput(pvname+".DESC", desc)
        epics.caput(pvname+".ZNAM", znam)
        epics.caput(pvname+".ONAM", onam)

plc_protect = SimPlcProtectionDevice(name="plc_protect")
# set text in the PVs to make the GUI look pretty
plc_protect.set_epics_names(plc_protect.SAXS_Y, "X11 SAXS_Y", "Low", "High")
plc_protect.set_epics_names(plc_protect.WAXS_X, "X12 WAXS_X", "Low", "High")
plc_protect.set_epics_names(plc_protect.AX, "X13 AX", "Low", "High")
plc_protect.set_epics_names(plc_protect.operations_status, "Y0 operations_status", "not good", "good")

In [4]:
import APS_BlueSky_tools.suspenders as APS_suspenders

class PlcProtectSuspendWhenChanged(APS_suspenders.SuspendWhenChanged):
    
    justification_text = """. 
    Significant equipment problem.  Do these steps:
    1. ^C twice      # interrupt the ipython kernel
    2. RE.abort()    # finalize current data streams (if any)
    3. exit          # quit bluesky session
    4. call beamline scientists

    """

    def _get_justification(self):
        """override default method to call plc_protect.stop_if_tripped()"""
        if not self.tripped:
            return ''

        self._tripped_message = None
        just = 'Signal {}, got "{}", expected "{}"'.format(
            self._sig.name,
            self._sig.get(),
            self.expected_value)
        if not self.allow_resume:
            just += self.justification_text
            self._tripped_message = plc_protect.stop_in_suspender()

        return '\n----\n'.join(s for s in (just, self._tripped_message) if s)

In [5]:
from APS_BlueSky_tools.devices import SimulatedApsPssShutterWithStatus
ti_filter_shutter = SimulatedApsPssShutterWithStatus(name="ti_filter_shutter")

class UserDataSimulator(Device):
    collection_in_progress = Component(EpicsSignal, "otz:vars0:bit4")
user_data = UserDataSimulator(name="user_data")

In [6]:
RE = RunEngine({})

# Important for routine operations
# see: https://github.com/APS-USAXS/ipython-usaxs/issues/82#issuecomment-444187217
suspend_plc_protect_operations_status = PlcProtectSuspendWhenChanged(
    plc_protect.operations_status, 
    expected_value=1)
# note: will fail if plc_protect.operations_status is "not good" - we want that!
RE.install_suspender(suspend_plc_protect_operations_status)
plc_protect.suspender = suspend_plc_protect_operations_status

In [7]:
det = EpicsSignal("otz:vars0:float1", name="det")

In [8]:
# suspend_plc_protect_operations_status.allow_resume = True
RE(bp.count([det], num=10, delay=1), LiveTable([det]))

+-----------+------------+------------+
|   seq_num |       time |        det |
+-----------+------------+------------+
|         1 | 13:02:01.3 |    0.00000 |
|         2 | 13:02:02.3 |    0.00000 |
|         3 | 13:02:03.3 |    0.00000 |
|         4 | 13:02:04.3 |    0.00000 |
Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2018-12-05 13:02:04.
Justification for this suspension:
Signal plc_protect_operations_status, got "0", expected "1". 
    Significant equipment problem.  Do these steps:
    1. ^C twice      # interrupt the ipython kernel
    2. RE.abort()    # finalize current data streams (if any)
    3. exit          # quit bluesky session
    4. call beamline scientists

    
----

    Equipment protection is engaged, no power on motors.
    Fix PLC protection before any move. Stopping now.
    Call beamline scientists if you do not understand.

    !!!!!!  DO NOT TRY TO FIX THIS YOURSELF  !!!!!!

    
 P.S. Can resume Bluesky scan: False

A 'defe

RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [9]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...
+-----------+------------+------------+
generator count ['7c8689e8'] (scan num: 1)


['7c8689e8-f49d-4ef1-826c-a9419c9eba4b']